### Capstone A - Data


The function `get_capstonedata_as_df()` returns email data in dataframe. The field `istestdata` column contains if the row is test data or train data. 

* `0` train data
* `1` test data

Include `%run CapstoneA.ipynb` as first line in your notebook and excucte. Then you can use `get_capstonedata_as_df()` anywhere in your notebook as local library.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from os.path import isfile
import email
#import BeautifulSoup
from bs4 import BeautifulSoup
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import silhouette_score
import hdbscan
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn import metrics as mt
from sklearn.feature_extraction import text
from wordcloud import WordCloud
from scipy.sparse import hstack

import warnings
import getpass
warnings.filterwarnings("ignore")



In [ ]:


def get_capstonedata_as_df():

    # location of emails
    username = getpass.getuser()
    if username == "taichowdhury":
        data_path = "/Users/taichowdhury/Documents/OneDrive/OneDrive - Southern Methodist University/CapstoneA/Data/phishingdata"
    elif username == "ravis":
        data_path = "/Users/"+username+"/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/phishingdata"
    elif username == "apurv":
        data_path = "/Users/apurv/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data"
        
    # get all sub folders
    sub_folders = [x[0] for x in os.walk(data_path) if x[0] != data_path]

    # read all emails from all sub folders
    mail_ty = []
    text_ty = []
    data = []
    target = []
    email_attachment = []
    attachment = False
    is_test_data = []

    for folder in sub_folders:
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        for file in files:
            with open(f"{folder}/{file}", encoding="latin1") as f:
                #    with open(f"{folder}/{file}","r") as f:
                x = email.message_from_file(f)
              #  print(x)
        #    if (file != 'cmds'): 
        #        mail_data.append(lines)

            mail_type = x.get_content_type()
            text_type = x.get_content_charset()
            mail_ty.append(mail_type)
            text_ty.append(text_type)
            if re.search("spam", folder):
                target.append(1)
            else:
                 target.append(0)

            if "testdata" in folder:
                is_test_data.append(1)
            else:
                is_test_data.append(0)

            if mail_type == "text/html":
                if not (isinstance(x.get_payload(), str)) and x.get_payload().get('Content-Disposition'):
                    dispositions = x.get_payload().get("Content-Disposition", None).strip().split(";")
                    if bool(dispositions[0].lower() == "attachment"):
                        attachment = True
                    else:
                        attachment = False

                tmp = BeautifulSoup(x.get_payload(), 'html.parser')
                tmp = tmp.text.replace("\n", " ")
                data.append(tmp)
            elif "multipart" in mail_type:
                attachment = False
                multipart_data = []
                for text in x.get_payload():                        
                    if not isinstance(text, str): 
                        if text.get('Content-Disposition'):
                            dispositions = text.get("Content-Disposition", None).strip().split(";")
                            if bool(dispositions[0].lower() == "attachment"):
                                attachment = True

                        if text.get_content_type() == "text/html":
                            tmp = BeautifulSoup(text.get_payload(), 'html.parser')
                            tmp = tmp.text.replace("\n", " ")
                            multipart_data.append(tmp)
                        elif text.get_content_type() == "text/plain":
                            multipart_data.append(text.get_payload()) 

                multipart_email = [''.join(str(item)) for item in multipart_data]
                data.append(multipart_email)
            else:
                if not (isinstance(x.get_payload(), str)) and x.get_payload().get('Content-Disposition'):
                    dispositions = x.get_payload().get("Content-Disposition", None).strip().split(";")
                    if bool(dispositions[0].lower() == "attachment"):
                        attachment = True
                    else:
                        attachment = False
                data.append(x.get_payload()) 

            if attachment:
                email_attachment.append(1)
            else:
                email_attachment.append(0)


    # Reference: https://gaurav.kuwar.us/index.php/2017/10/09/extracting-files-from-raw-email-with-python/

    df = pd.DataFrame()
    df["mail_types"] = mail_ty
    df["text_types"] = text_ty

    # Create a data frame with email text and target (whether mail is spam or not, 1 for spam and 0 for not a spam)
    email_df = pd.DataFrame()
    email_df["data"] = data
    #email_df["mail_type"] = mail_ty
    #email_df["text_type"] = text_ty
    email_df["target"] = target
    email_df["Attachments"] = email_attachment
    email_df["istestdata"] = is_test_data
    
    return email_df
